# Prepare necessary packages and information

In [ ]:
!pip install -r requirements.txt

In [ ]:
!pip uninstall -y ninja && pip install ninja
!ninja --version

In [ ]:
!pip install flash-attn --no-build-isolation
!git clone https://github.com/Dao-AILab/flash-attention
%cd flash-attention && pip install .

In [ ]:
import torch
torch.cuda.is_available()

In [ ]:
model_id = "Qwen/Qwen-7B-Chat-Int4"
train_data_path = "dataset/human_qwen_train_output.json"
valid_data_path = "dataset/human_qwen_valid_output.json"

In [ ]:
# Make sure you are in the project root
%pwd

# Inference without finetuning

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", trust_remote_code=True).eval()
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)

In [ ]:
history = None
question = 'You are a code assistant of the SSDP platform which is an internal tool used to create 3D scenes. In the SSDP platform, write a function to create a line segment from point (1, 2, 3) to (3, 2, 1) and add it to the viewport. Directly give the code here:'

response, history = model.chat(tokenizer, question, history=history)
print(response)

# Check the resources of finetuning

In [ ]:
torch.cuda.empty_cache()
!nvidia-smi

# Finetune

In [ ]:
# Finetune with q-lora on multiple GPUs
!bash finetune/finetune_qlora_ds.sh -m model_id -d train_data_path -e valid_data_path

In [ ]:
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer

model = AutoPeftModelForCausalLM.from_pretrained(
    "checkpoints",
    device_map="auto",
    trust_remote_code=True
).eval()

tokenizer = AutoTokenizer.from_pretrained("checkpoints", trust_remote_code=True)

# Evaluate the result

In [ ]:
history = None
question = 'You are a code assistant of the SSDP platform which is an internal tool used to create 3D scenes. In the SSDP platform, write a function to create a line segment from point (1, 2, 3) to (3, 2, 1) and add it to the viewport. Directly give the code here:'

response, history = model.chat(tokenizer, question, history=history)
print(response)

In [ ]:
question = 'You are a code assistant of the SSDP platform which is an internal tool used to create 3D scenes. In the SSDP platform, write a function to create a line segment from point (0, 0, 0) to (1, 1, 1) and add it to the viewport. Directly give the code here:'
response, history = model.chat(tokenizer, question, history=history)
print(response)

In [ ]:
question = 'You are a code assistant of the SSDP platform which is an internal tool used to create 3D scenes. In the SSDP platform, write a function to create a 2d circle with the center(1, 1, 1) , radius 2 and normal vector Z-axis. Directly give the code here:'
response, history = model.chat(tokenizer, question, history=history)
print(response)